In [1]:
import pip
           
def import_or_install(package):
    try:
        exec("import {module}".format(module=package))
        # import os
        # use os to pip install library
    except ImportError:
        pip.main(['install', package])         

list_of_modules = ['csv','requests','openpyxl','os','pandas','datetime','bs4','openpyxl','datetime','tqdm','io','sklearn']
for module in list_of_modules:
    import_or_install(module)

import csv
import requests
import openpyxl
import os
import pandas as pd
import datetime
import bs4 
import openpyxl.styles 
import datetime 
import tqdm 
import openpyxl.chart 
import io
import sklearn.linear_model

from bs4 import BeautifulSoup
from openpyxl.styles import Font, Border, Side
from datetime import date, timedelta
from tqdm import tqdm
from openpyxl.chart import LineChart, Reference, Series
from io import StringIO
from sklearn.linear_model import LinearRegression

thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

print('Uploading data in progress, please wait!')


### BELOW CODE (CLASS) WILL UPLOAD NEW DATA TO THE FILE
def find_companies(dict_name,list_to):
    # create dict with links to wig20 companies historical data + names of sheets
    

    #source = requests.get('https://stooq.pl/t/?i=532').content
    #soup = BeautifulSoup(source, 'lxml')

    #names = soup.find_all('td', attrs={'id': 'f13'})
    #names_of_companies = [names[1 + num * 6].text for num in range(0, 20)]
    
    link = 'https://stooq.pl/q/d/l/?s='
    link2 = '&i=d'

    for i in list_to:
        dict_name[link + i.lower() + link2] = i
    

def find_companies2(dict_name2, gowno):
    # create dict with links to wig20 companies indicators + names of sheets
    source = requests.get('https://www.biznesradar.pl/gielda/indeks:WIG20').content
    soup = BeautifulSoup(source, 'lxml')

    names = soup.find_all('td')

    names_of_companies = [names[i * 13].text.split(" ")[0] for i in range(0, 20)]
    names_of_companies.append("KTY")

    link = 'https://www.biznesradar.pl/wskazniki-rentownosci/'
    
    for i in names_of_companies:
        list_to.append(i.lower())
        
    
        
    list_to = names_of_companies
    for i in names_of_companies:
        dict_name2[link + i] = i


def create_file(sheet_names):
    path = os.path.expanduser("~/Desktop")

    wb = openpyxl.Workbook()

    for i in sheet_names:
        wb.create_sheet(i)

    wb.remove(wb['Sheet'])

    wb.save(path + '/Wig20_analysis.xlsm')


class upload_data:
    def __init__(self, link, sheet):
        self.link = link
        self.sheet = sheet

    def download_data(self):
        # download data and calculate the % diffirance for Zamkniecie and add colour if the value is >0 or <0
        with requests.Session() as downloaded_file:
            download = downloaded_file.get(self.link)

            decoded_content = download.content.decode('utf-8')

            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            my_list = list(cr)
            for j in range(0, len(my_list)):
                for i in range(0, len(my_list[j])):
                    if j > 0:
                        try:
                            if i > 0:
                                self.sheet.cell(row=j + 1, column=i + 1).value = float(my_list[::-1][j - 1][i])
                                self.sheet.cell(row=j + 1, column=i + 1).border = thin_border

                            else:
                                self.sheet.cell(row=j + 1, column=i + 1).value = datetime.datetime.strptime(
                                    my_list[::-1][j - 1][i], '%Y-%m-%d').strftime('%d/%m/%Y')
                                self.sheet.cell(row=j + 1, column=i + 1).border = thin_border
                        except IndexError:
                            self.sheet.cell(row=j + 1, column=i + 1).value = 'no data'
                            self.sheet.cell(row=j + 1, column=i + 1).border = thin_border
                    else:
                        self.sheet.cell(row=j + 1, column=i + 1).value = my_list[j][i]
                        self.sheet.cell(row=j + 1, column=i + 1).border = thin_border

                self.sheet.cell(row=j + 2, column=8).value = str(self.sheet)

            self.sheet.cell(row=1, column=8).value = 'Tableau'
            self.sheet.cell(row=11, column=9).value = 'Data wejścia na giełdę:'
            self.sheet.cell(row=11, column=10).value = my_list[1][0]

            for i in range(1, len(my_list) - 1):
                self.sheet.cell(row=i + 1, column=7).value = 1 - float(my_list[::-1][i][4]) / float(
                    my_list[::-1][i - 1][4])
                self.sheet.cell(row=i + 1, column=7).border = thin_border
                if 1 - float(my_list[::-1][i][4]) / float(my_list[::-1][i - 1][4]) > 0 and i > 0:
                    self.sheet.cell(row=i + 1, column=7).font = Font(color="FF00FF00")
                    self.sheet.cell(row=i + 1, column=7).number_format = '0.00%'
                elif 1 - float(my_list[::-1][i][4]) / float(my_list[::-1][i - 1][4]) < 0 and i > 0:
                    self.sheet.cell(row=i + 1, column=7).font = Font(color="FFFF0000")
                    self.sheet.cell(row=i + 1, column=7).number_format = '0.00%'

            self.sheet.cell(row=1, column=7).value = "Cena zamkniecia - zmiana [%]"
            self.sheet.cell(row=1, column=7).border = thin_border

    def statistics(self):
        ### create table with statistics data (Q1,mean,std ETC)
        r = requests.get(self.link)
        df = pd.read_csv(StringIO(r.text))

        list_of_headers = ['Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen']
        names = ['Suma', 'Średnia', 'Standardowe odchylenie', 'Wartość minimalna', 'Q1', 'Q2', 'Q3',
                 'Wartość maksymalna']

        # add headers
        for col_num in range(0, len(list_of_headers)):
            self.sheet.cell(row=1, column=col_num + 10).value = list_of_headers[col_num]
            self.sheet.cell(row=1, column=col_num + 10).border = thin_border
        # add names of rows
        for row_num in range(0, len(names)):
            self.sheet.cell(row=row_num + 2, column=9).value = names[row_num]
            self.sheet.cell(row=row_num + 2, column=9).border = thin_border

        # add values
        for col_num in range(0, len(list_of_headers)):
            for row_num in range(0, 8):
                self.sheet.cell(row=row_num + 2, column=col_num + 10).value = \
                df[list_of_headers[col_num]].describe().tolist()[row_num]
                self.sheet.cell(row=row_num + 2, column=col_num + 10).border = thin_border

        ### CREATE LINEAR REGRESION MODEL TO PREDICT FUTURE DATA
        # prediction
        self.sheet.cell(row=1, column=17).value = 'Analiza Regresji Liniowej'
        self.sheet.cell(row=2, column=17).value = 'Wzór modelu funkcji'
        self.sheet.cell(row=4, column=17).value = 'Data'
        self.sheet.cell(row=5, column=17).value = 'Przewidywane wartości'
        self.sheet.cell(row=6, column=17).value = 'Dopasowanie modelu'

        lm = LinearRegression()

        X = df[['Otwarcie', 'Najwyzszy', 'Najnizszy']][::-1]
        y = df['Zamkniecie'][::-1]

        # create a model
        lm = LinearRegression()
        model = lm.fit(X, y)

        # coefficient Zamkniecie
        a = lm.intercept_

        # coefficient Otwarcie,Najwyzszy,Najnizszy
        b = lm.coef_.tolist()

        # create formula
        function_formula = str(a) + ' × ' + str(b[0]) + ' + ' + str(a) + ' × ' + str(b[1]) + ' + ' + str(
            a) + ' × ' + str(b[2])
        self.sheet.cell(row=2, column=18).value = function_formula

        # predict future value
        predicted_value = lm.predict(X)[0:5].tolist()

        # model fit
        model_evaluation = lm.score(X, y)

        # past values in excel file
        days = 0
        tomorrow = date.today() + timedelta(days=1)
        for i in range(0, 5):
            self.sheet.cell(row=5, column=18 + i).value = predicted_value[i]
            self.sheet.cell(row=5, column=18 + i).border = thin_border
            if (tomorrow + timedelta(days=i)).weekday() not in [5, 6]:
                self.sheet.cell(row=4, column=18 + i).value = tomorrow + timedelta(days=i + days)
            elif (tomorrow + timedelta(days=i)).weekday() == 5:
                self.sheet.cell(row=4, column=18 + i).value = tomorrow + timedelta(days=i + 2)
                days = 2
            elif (tomorrow + timedelta(days=i)).weekday() == 6:
                self.sheet.cell(row=4, column=18 + i).value = tomorrow + timedelta(days=i + 2)
                days = 2

        self.sheet.cell(row=6, column=18).value = model_evaluation

    def create_charts(self, name, last_row, where1, where2):

        opening = Reference(self.sheet, min_col=2, min_row=2,
                            max_col=2, max_row=last_row)

        highest = Reference(self.sheet, min_col=3, min_row=2,
                            max_col=3, max_row=last_row)

        lowest = Reference(self.sheet, min_col=4, min_row=2,
                           max_col=4, max_row=last_row)

        closing = Reference(self.sheet, min_col=5, min_row=2,
                            max_col=5, max_row=last_row)

        volumen = Reference(self.sheet, min_col=6, min_row=2,
                            max_col=6, max_row=last_row)

        dates = Reference(self.sheet, min_col=1, min_row=2,
                          max_col=1, max_row=last_row)
        ## first chart
        chart = LineChart()
        series = Series(opening, title="Cena otwarcia")
        chart.append(series)
        series = Series(highest, title="Cena najwyższa")
        chart.append(series)
        series = Series(lowest, title="Cena najniższa")
        chart.append(series)
        series = Series(closing, title="Cena zamknięcia")
        chart.append(series)

        chart.set_categories(dates)
        chart.title = str(name)
        chart.x_axis.title = "Data"
        chart.y_axis.title = "Wartość"
        self.sheet.add_chart(chart, where1)  # h24
        chart.height = 10  # default is 7.5
        chart.width = 30  # default is 15
        ##

        ## second chart
        chart2 = LineChart()
        chart2.append(Series(volumen, title="Wolumen"))
        chart2.set_categories(dates)

        chart2.title = str(name)
        chart2.x_axis.title = "Data"
        chart2.y_axis.title = "Wartość"
        self.sheet.add_chart(chart2, where2)  # h45
        chart2.height = 10  # default is 7.5
        chart2.width = 30  # default is 15
        ##

    def adjusting_width(self):

        # add new columsn so it will be more transparent
        self.sheet.insert_cols(9)

        list_of_cols = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'S', 'T', 'U', 'W', 'V']
        list_of_cols2 = ['G', 'J', 'R']

        for letter in list_of_cols:
            self.sheet.column_dimensions[letter].width = 13

        for letter in list_of_cols2:
            self.sheet.column_dimensions[letter].width = 25

    def visual(self):

        for i in range(1, 6):
            self.sheet.cell(row=i, column=18).border = thin_border
            self.sheet.cell(row=4, column=18 + i).border = thin_border

        self.sheet.cell(row=6, column=18).border = thin_border
        self.sheet.cell(row=6, column=19).border = thin_border

        self.sheet.cell(row=11, column=10).border = thin_border
        self.sheet.cell(row=11, column=11).border = thin_border


class indicators:
    def __init__(self, link, sheet):
        self.link = link
        self.sheet = sheet

    def financial_indicators(self):

        source = requests.get(self.link).content
        soup = BeautifulSoup(source, 'lxml')

        indicator = soup.find_all('td', attrs={'class': 'h'})
        dates = soup.find_all('th', attrs={'class': 'thq h'})
        newest_date = soup.find_all('th', attrs={'class': 'thq h newest'})
        names_of_indicators = soup.find_all('td', attrs={'class': 'f'})

        row_num = 14
        for num in range(0, len(indicator)):
            if num % (len(dates) + len(newest_date)) != 0:

                if indicator[num].text == '':
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).value = 'N/A'
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).border = thin_border
                else:
                    b = indicator[num].text
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).value = str(b)[0:(
                                                                                                                         b.find(
                                                                                                                             '%')) + 1]
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).border = thin_border

            elif num % (len(dates) + 1) == 0:
                if num == 0:
                    pass
                else:
                    row_num += 1

                if indicator[num].text == '':
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).value = 'N/A'
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).border = thin_border
                else:
                    b = indicator[num].text
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).value = str(b)[0:(
                                                                                                                         b.find(
                                                                                                                             '%')) + 1]
                    self.sheet.cell(row=row_num, column=10 + num % (len(dates) + len(newest_date))).border = thin_border

                    # add dates
        for num in range(0, len(dates)):
            self.sheet.cell(row=13, column=10 + num).value = " ".join(dates[num].text.split())
            self.sheet.cell(row=13, column=10 + num).border = thin_border

        self.sheet.cell(row=13, column=10 + len(dates)).value = " ".join(newest_date[0].text.split())
        self.sheet.cell(row=13, column=10 + len(dates)).border = thin_border

        # add indicators names
        for num in range(0, len(names_of_indicators)):
            self.sheet.cell(row=14 + num, column=9).value = names_of_indicators[num].text
            self.sheet.cell(row=14 + num, column=9).border = thin_border


def main():
    dict_name2 = {}
    gowno = []
    find_companies2(dict_name2,gowno)
    indicators_for_companies = dict_name2
    
    dict_name = {}
    find_companies(dict_name,gowno)
        
    list_of_companies = dict_name
    path = os.path.expanduser("~/Desktop")

    # remove file if it already exists
    try:
        os.remove(path + "/Wig20_analysis.xlsm")
    except FileNotFoundError:
        pass
    # create new file
    create_file(list(list_of_companies.values()))
    working_file = openpyxl.load_workbook(filename=path + '/Wig20_analysis.xlsm', read_only=False, keep_vba=True)

    # for ws in working_file:
    #    ws.column_dimensions['H'].hidden = False

    for num in tqdm(range(0, len(list(list_of_companies.keys())))):
        company = upload_data(list(list_of_companies.keys())[num], working_file[list(list_of_companies.values())[num]])
        companys_indicators = indicators(list(indicators_for_companies.keys())[num],
                                         working_file[list(list_of_companies.values())[num]])
        company.download_data()

        company.statistics()

        companys_indicators.financial_indicators()

        company.create_charts(list_of_companies[list(list_of_companies.keys())[num]],
                              len(working_file[list(list_of_companies.values())[num]]['A']), "J24", "J45")

        company.create_charts(list_of_companies[list(list_of_companies.keys())[num]], 14, "V24", "V45")

        company.adjusting_width()

        company.visual()

    for ws in working_file:
        ws.column_dimensions['H'].hidden = True
    working_file.save(path + '/Wig20_analysis.xlsm')

    print('Process completed!')


if __name__ == "__main__":
    main()

Uploading data in progress, please wait!


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [03:08<00:00,  8.99s/it]


Process completed!
